In [176]:
import json
import math
import numpy as np

In [177]:
# finds reviewed protein's domain positions in interpro JSON by accession string
# returns None, if protein with given accession wasn't found in interpro JSON
def find_protein_positions_by_accession(accession, interpro_data):
    for i in range(len(interpro_data)):
        if interpro_data[i]['metadata']['accession'] == accession and interpro_data[i]['metadata']['source_database'] != "unreviewed":
            positions = []
            
            for position in interpro_data[i]['entries'][0]['entry_protein_locations']:
                positions.append((position['fragments'][0]['start'], position['fragments'][0]['end']))
            
            return positions
    
    return None

In [178]:
'''
Example cases (from HMM model):

# Model: [(744, 806), (917, 970), (1006, 1057), (1082, 1138), (1162, 1212)]
# Reference: [(919, 963), (1008, 1052), (1163, 1206)]

# Model: [(506, 550)]
# Reference: [(507, 563)]
'''

def compute_metrics(model_hits, reference_hits):
    leftmost_pos = model_hits[0][0] if model_hits[0][0] < reference_hits[0][0] else reference_hits[0][0]
    rightmost_pos = model_hits[-1][1] if model_hits[-1][1] > reference_hits[-1][1] else reference_hits[-1][1]
    model_arr, ref_arr = np.zeros(rightmost_pos - leftmost_pos), np.zeros(rightmost_pos - leftmost_pos)
    
    # put 1 in ref_arr where PF00018 domain is present
    for reference_hit in reference_hits:
        ref_arr[reference_hit[0] - leftmost_pos:reference_hit[1] - leftmost_pos] = 1
    
    # put 2 in model_arr where hit was found
    for model_hit in model_hits:
        model_arr[model_hit[0] - leftmost_pos:model_hit[1] - leftmost_pos] = 2
    
    result = model_arr + ref_arr
    
    tp = np.sum(result == 3)
    tn = np.sum(result == 0)
    fp = np.sum(result == 2)
    fn = np.sum(result == 1)
    
    return np.array([tp, tn, fp, fn])

In [179]:
with open("interpro-pfam-data.json", "r") as read_file:
    interpro_data = json.load(read_file)

In [180]:
# Analyze HMM model

hmm_results = open("hmmresults/hmmsearch.hmmer_domtblout")

# skip unused lines
hmm_results.__next__()
hmm_results.__next__()
hmm_results.__next__()

first_hit = hmm_results.__next__().split()

current_protein_id = first_hit[0].split("|")[1]
current_protein_hits = [(int(first_hit[17]), int(first_hit[18]))]

# accumulates tp, tn, fp, fn values from all the proteins
metrics_accum = np.zeros(4)

for line in hmm_results:
    line_data = line.split()
    
    # last line
    if len(line_data) == 1:
        # compute metrics for the last protein
        metrics_accum += compute_metrics(current_protein_hits, find_protein_positions_by_accession(current_protein_id, interpro_data))
        break
    
    protein_id = line_data[0].split("|")[1]
    
    if protein_id != current_protein_id:        
        # Calculation of TP, TN, FP, FN metrics for the given protein
        metrics_accum += compute_metrics(current_protein_hits, find_protein_positions_by_accession(current_protein_id, interpro_data))
        
        current_protein_id = protein_id
        current_protein_hits = []
    
    hit_start = int(line_data[17])
    hit_end = int(line_data[18])
    
    current_protein_hits.append((hit_start, hit_end))


tp = metrics_accum[0]
tn = metrics_accum[1]
fp = metrics_accum[2]
fn = metrics_accum[3]
mcc = (tp * tn - fp * fn) / math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

In [181]:
print("Results for the HMM model:")
print(f"Number of True Positives: {int(tp)}")
print(f"Number of True Negatives: {int(tn)}")
print(f"Number of False Positives: {int(fp)}")
print(f"Number of False Negatives: {int(fn)}")
print(f"MCC score: {mcc}")

Results for the HMM model:
Number of True Positives: 5377
Number of True Negatives: 5968
Number of False Positives: 2895
Number of False Negatives: 55
MCC score: 0.6519685391994191


In [182]:
# Analyze PSSM model

pssm_results = open("pssmresults/psiblast_out_tabular.txt")

first_hit = pssm_results.__next__().split()

current_protein_id = first_hit[1]
current_protein_hits = [(int(first_hit[8]), int(first_hit[9]))]

# accumulates tp, tn, fp, fn values from all the proteins
metrics_accum = np.zeros(4)

for line in pssm_results:
    line_data = line.split()
    
    protein_id = line_data[1]
    
    if protein_id != current_protein_id:        
        # Calculation of TP, TN, FP, FN metrics for the given protein
        metrics_accum += compute_metrics(current_protein_hits, find_protein_positions_by_accession(current_protein_id, interpro_data))
        
        current_protein_id = protein_id
        current_protein_hits = []
    
    hit_start = int(line_data[8])
    hit_end = int(line_data[9])
    
    current_protein_hits.append((hit_start, hit_end))

# Last protein
metrics_accum += compute_metrics(current_protein_hits, find_protein_positions_by_accession(current_protein_id, interpro_data))

tp = metrics_accum[0]
tn = metrics_accum[1]
fp = metrics_accum[2]
fn = metrics_accum[3]
mcc = (tp * tn - fp * fn) / math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

In [183]:
print("Results for the PSSM model:")
print(f"Number of True Positives: {int(tp)}")
print(f"Number of True Negatives: {int(tn)}")
print(f"Number of False Positives: {int(fp)}")
print(f"Number of False Negatives: {int(fn)}")
print(f"MCC score: {mcc}")

Results for the PSSM model:
Number of True Positives: 4433
Number of True Negatives: 4298
Number of False Positives: 517
Number of False Negatives: 610
MCC score: 0.7714680429593606
